### It is necessary to change path of the database located in the github folder /data

In [1]:
import numpy as np
import pandas as pd
import os


column_names = ['patient_number', 'age', 'sex', 'adult_BMI', 'child_weight', 'child_height']
dataset = pd.read_csv('../input/respiratory-sound-database/demographic_info.txt', sep=" ", header=None, names =column_names)
dataset['sex'] = dataset['sex'].replace({'F': 1, 'M': 0})


column_names = ['Beginning of respiratory cycle(s)','End of respiratory cycle(s)','Crackles','Wheezes']
patient_dataset = {}

for file in os.listdir("../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files"):
        if file.endswith(".txt"):
            patient_number = file.split("_")[0]
            data = pd.read_csv('../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/'+file, sep="\t", header=None, names = column_names)
            for crackles in data['Crackles']:
                if crackles == 1:
                    if(patient_dataset.get(patient_number+"_crackles") != None):
                        patient_dataset[patient_number+"_crackles"] += 1
                    else:
                        patient_dataset[patient_number+"_crackles"] = 1;
            for wheezes in data['Wheezes']:
                if wheezes == 1:
                    if(patient_dataset.get(patient_number+"_wheezes") != None):
                        patient_dataset[patient_number+"_wheezes"] += 1
                    else:
                        patient_dataset[patient_number+"_wheezes"] = 1;
            
final_patient_dataset = []

new_data = {#'patient_number': [pd.to_numeric(101)], 
               'n_wheezes': [patient_dataset.get(str(101) + ("_wheezes"))],
               'n_crackles': [patient_dataset.get(str(101) + ("_crackles"))]}
new_data = pd.DataFrame(new_data)
final_strings_for_dataset = pd.DataFrame(new_data)


for patient_number in range (102, 227):  
    
    new_data = {#'patient_number': [pd.to_numeric(patient_number)], 
               'n_wheezes': [patient_dataset.get(str(patient_number) + ("_wheezes"))],
               'n_crackles': [patient_dataset.get(str(patient_number) + ("_crackles"))]}
    new_data = pd.DataFrame(new_data)
    final_strings_for_dataset = final_strings_for_dataset.append([new_data],  ignore_index=True)


final_strings_for_dataset.loc[~final_strings_for_dataset.index.duplicated(keep='first')]
dataset.loc[~dataset.index.duplicated(keep='first')]

dataset = pd.concat([dataset, final_strings_for_dataset], axis=1)
dataset = dataset.replace(np.nan, '0')


Y = pd.read_csv('../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv', sep=",", header=None, names=['patient_number', 'diagnosis'])
Y = Y['diagnosis']

'''
['URTI' 'Healthy' 'Asthma' 'COPD' 'LRTI' 'Bronchiectasis' 'Pneumonia' 'Bronchiolitis']
'''

Y = Y.replace('Healthy', 0)
Y = Y.replace('URTI', 1)
Y = Y.replace('Asthma', 1)
Y = Y.replace('COPD', 1)
Y = Y.replace('LRTI', 1)
Y = Y.replace('Bronchiectasis', 1)
Y = Y.replace('Pneumonia', 1)
Y = Y.replace('Bronchiolitis', 1)
Y = Y.replace(np.nan, '0')

Y = pd.DataFrame(Y)
print(dataset)

FileNotFoundError: [Errno 2] No such file or directory: '../input/respiratory-sound-database/demographic_info.txt'

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from matplotlib.legend_handler import HandlerLine2D
import seaborn as sn
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'sklearn.datasets.samples_generator'

In [3]:
dataset_train = dataset
dataset_train = dataset_train.drop(columns='patient_number')
dataset_train = dataset_train.drop(columns='adult_BMI')
dataset_train = dataset_train.drop(columns='child_weight')
dataset_train = dataset_train.drop(columns='child_height')
print(dataset_train)

NameError: name 'dataset' is not defined

In [4]:
x_train, x_test, y_train, y_test = train_test_split(dataset_train, np.ravel(Y), test_size=0.25)

max_depths = np.linspace(1, 30, 30, endpoint=True)
train_results = []
test_results = []
train_errors =[]
test_errors = []

for max_depth in max_depths:
    clf = DecisionTreeClassifier(max_depth=max_depth)
    clf.fit(x_train, y_train)
    train_pred = clf.predict(x_train)
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    
    error = sum(train_pred != y_train)/len(y_train)
    train_results.append(roc_auc)
    train_errors.append(error)
    
    '''
    if(max_depth < 8 and max_depth > 3):
        print("Tree depth: ", max_depth)
        print("Train error: ", 1-error)
    '''
    
    y_pred = clf.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    error = sum(y_pred != y_test)/len(y_test)
    
    test_errors.append(error)
    test_results.append(roc_auc)
    
    '''
    if(max_depth < 8 and max_depth > 3):
        print("Test error: ", 1-error)
    '''

plt.rcParams['figure.figsize'] = [15, 15]
plt.subplot(221)
line1, = plt.plot(max_depths, train_errors, 'b', label="Train error")
line2, = plt.plot(max_depths, test_errors, 'r', label="Test error")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('Error', size = 20)
plt.xlabel('Tree depth', size = 20)
plt.legend(prop={'size': 15})
plt.title('Depth Tree Analysis', size = 25)


plt.subplot(222)
line1, = plt.plot(max_depths, train_results, 'b', label="Train AUC")
line2, = plt.plot(max_depths, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('Accuracy', size = 20)
plt.xlabel('Tree depth', size = 20)
plt.legend(prop={'size': 15})
plt.title('Depth Tree Analysis', size = 25)


plt.show()

NameError: name 'dataset_train' is not defined

## CHILDREN AND ADULTS ANALYSIS

In [5]:
column_names = ['patient_number', 'age', 'sex', 'adult_BMI', 'child_weight', 'child_height']
dataset = pd.read_csv('../input/respiratory-sound-database/demographic_info.txt', sep=" ", header=None, names =column_names)
dataset['sex'] = dataset['sex'].replace({'F': 1, 'M': 0})

dataset['age'] = dataset['age'].astype(float)
dataset_adult = dataset
adults = np.where(dataset['age']>18)

dataset_adults = pd.DataFrame()
dataset_children = pd.DataFrame()
labels_adults = pd.DataFrame()
labels_children = pd.DataFrame()


for x in range(1,126):
    if(np.any(adults[0] == x)):
        new_data = dataset.iloc[[x]]
        new_label = Y.iloc[[x]]
        dataset_adults = dataset_adults.append(new_data)
        labels_adults = labels_adults.append(new_label)
    else:
        new_data = dataset.iloc[[x]]
        new_label = Y.iloc[[x]]
        dataset_children = dataset_children.append(new_data)
        labels_children = labels_children.append(new_label)
    


FileNotFoundError: [Errno 2] No such file or directory: '../input/respiratory-sound-database/demographic_info.txt'

In [7]:
print(dataset_children.shape)
print(dataset_adults.shape)
#print(labels_children.shape)
#print(labels_adults.shape)

dataset_children = dataset_children.drop(columns='adult_BMI')
dataset_children = dataset_children.drop(columns='patient_number')

dataset_adults = dataset_adults.drop(columns='child_weight')
dataset_adults = dataset_adults.drop(columns='child_height')
dataset_adults = dataset_adults.drop(columns='patient_number')
#dataset_adults = dataset_adults.drop(columns='age')
#dataset_adults = dataset_adults.drop(columns='sex')

dataset_children = dataset_children.replace(np.nan, '0')
dataset_adults = dataset_adults.replace(np.nan, '0')


print(dataset_adults)
print(dataset_children)

NameError: name 'dataset_children' is not defined

In [ ]:
# No classification possible, only one label is given (Every adult is sick!)
print(np.where(labels_adults==0))